In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=255e01af952b83a7408ce3c2da3e7762d9d35443ce0f2f54a9d1d73b4e777abc
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lrex").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
training = spark.read.format("libsvm").load('/content/drive/MyDrive/Colab Notebooks/data/sample_linear_regression_data.txt')

In [4]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [9]:
training_summary = lrModel.summary

In [10]:
training_summary.r2

0.027839179518600154

In [11]:
training_summary.r2adj

0.007999162774081858

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [13]:
training_summary.residuals

DataFrame[residuals: double]

#### Splitting Data into Training and Test Data

In [14]:
all_data = spark.read.format("libsvm").load('/content/drive/MyDrive/Colab Notebooks/data/sample_linear_regression_data.txt')

In [15]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
|-15.951512565794573|(10,[0,1,2,3,4,5,...|
+----------

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                340|
|   mean|-0.1062283033488525|
| stddev|  10.16737663482349|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [18]:
correct_model = lr.fit(train_data)

In [19]:
correct_model_summary = correct_model.summary

In [20]:
correct_model_summary.r2

0.039820079460322955

In [21]:
test_results = correct_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-29.012596474681057|
| -22.55085123779704|
| -19.41874362487324|
| -21.40452700016254|
|-18.979380847113692|
| -16.50293791386379|
|-15.357783198637536|
| -19.51631330796328|
|-16.609260168702342|
|-17.793552906099475|
|-14.805921875407412|
|-12.980908197688944|
|-13.935128342887424|
|-17.462432806352925|
|-14.457656622461686|
| -12.37778515727035|
| -8.450962528403606|
|-13.965615437659439|
|-15.257505026166141|
|-13.560783520220419|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

10.834361239605633

In [24]:
unlabeled_data = test_data.select('features')

In [25]:
unlabeled_data.show(4)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 4 rows



In [26]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   2.207113046197986|
|(10,[0,1,2,3,4,5,...| -0.9365888831394729|
|(10,[0,1,2,3,4,5,...| -0.3640191647412974|
|(10,[0,1,2,3,4,5,...|    1.73720838479082|
|(10,[0,1,2,3,4,5,...|-0.42295518310086216|
|(10,[0,1,2,3,4,5,...| -2.6653547090999714|
|(10,[0,1,2,3,4,5,...| -1.9689375340384119|
|(10,[0,1,2,3,4,5,...|  2.4509136820872635|
|(10,[0,1,2,3,4,5,...|-0.41723209550720614|
|(10,[0,1,2,3,4,5,...|  1.1013458847883706|
|(10,[0,1,2,3,4,5,...| -1.2797371656140772|
|(10,[0,1,2,3,4,5,...| -2.4564765957422727|
|(10,[0,1,2,3,4,5,...| -1.4407293804248726|
|(10,[0,1,2,3,4,5,...|   3.133454297277482|
|(10,[0,1,2,3,4,5,...| 0.48152569130898243|
|(10,[0,1,2,3,4,5,...| -0.1014950541811474|
|(10,[0,1,2,3,4,5,...|  -3.959983874436554|
|(10,[0,1,2,3,4,5,...|   1.871264159124181|
|(10,[0,1,2,3,4,5,...|  3.4304320297735695|
|(10,[0,1,2,3,4,5,...|  2.614863

#### Regression Evaluation 

1. Mean Absolute Error
2. Mean Squared Error
3. Root Mean Squared Error
4. R Square value

#### Real Time Project on Linear Regression 

In [27]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lr_example").getOrCreate()
from pyspark.ml.regression import LinearRegression
data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/data/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [28]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [29]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [30]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [31]:
#### Setup data sets for Machine Learning 
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [32]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [33]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')


In [34]:
output = assembler.transform(data)

In [35]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [36]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [37]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [38]:
final_data = output.select('features','Yearly Amount Spent')

In [39]:
final_data.show(4)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
+--------------------+-------------------+
only showing top 4 rows



In [41]:
train_data,test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean| 497.93825590983846|
| stddev|  82.90635824615104|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [42]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean| 502.52419707234657|
| stddev|  70.38147219939536|
|    min| 302.18954780965197|
|    max|  744.2218671047146|
+-------+-------------------+



In [43]:
lr = LinearRegression(labelCol='Yearly Amount Spent')
lr_model = lr.fit(train_data)

In [44]:
test_results = lr_model.evaluate(test_data)

In [45]:
test_results.rootMeanSquaredError

9.692401980806979

In [46]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.062192293416729|
| -2.238247633250637|
|  7.702098567565827|
|  -7.27414802959629|
| 22.862092561918416|
| -3.533187445052363|
|-5.2545369533646635|
|-2.6707371391592574|
| -8.539702816521299|
|-1.5910956422596882|
|-1.0848968002305241|
|  -8.74104933765642|
| -8.623115727841821|
|-17.814844901136667|
|   4.28933487980396|
|  3.103128389027461|
| 18.174484673120958|
|  4.569247096306299|
|0.13706970240014016|
|  22.19171582472427|
+-------------------+
only showing top 20 rows



In [47]:
test_results.r2

0.9809080129033126

In [48]:
test_results.r2adj

0.9803813373971971

In [49]:
unlabeled_data = test_data.select('features')

In [50]:
unlabeled_data.show(4)

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.8364326747734...|
|[30.9716756438877...|
|[31.1280900496166...|
+--------------------+
only showing top 4 rows



In [51]:
predictions = lr_model.transform(unlabeled_data)

In [52]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.57815877921075|
|[30.8364326747734...|469.74014806024024|
|[30.9716756438877...| 486.9365111893269|
|[31.1280900496166...|  564.526834776651|
|[31.2834474760581...|  568.918996863749|
|[31.4252268808548...| 534.2999060998143|
|[31.5257524169682...|449.22016376324655|
|[31.6253601348306...|379.00763789608345|
|[31.7207699002873...| 547.3146362945442|
|[31.8124825597242...| 394.4014406260569|
|[31.8186165667690...| 447.5035701703662|
|[31.8279790554652...|448.74379688459794|
|[31.8854062999117...|398.72638870031733|
|[31.9563005605233...| 564.9407766483355|
|[32.0305497162129...| 589.9851485388078|
|[32.0444861274404...| 445.1267007975225|
|[32.0478146331398...|479.21507308572245|
|[32.0542618511847...| 557.3054105726767|
|[32.0705462209254...| 532.6147178794363|
|[32.1253868972878...| 435.6559801202243|
+--------------------+------------